In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from os import listdir
from os.path import isfile, join
%matplotlib inline

repo = os.path.join('/home/sditom/us_hindecast_project/public')

In [2]:
### This script merges harmonics and weather variables of all grid points and all years to produce complete DataFrame

## Steps:
## 1)Load harmonics and drop NaN (harmonics, cropland-CDL)
## 2)Load Weather and drop NaN
## 3)Merge harmonics and Weather
## 4)Drop non-crop points
## 5)Add Target column (CDL)
## 6)Drop points at state borders and add state name column
## 7)Save DF

### 1) Harmonics

In [3]:
years = list(range(1999,2019))

In [4]:
def init_df(folder):
    
    files = [f for f in listdir(folder) if (isfile(join(folder, f)) and f.endswith('.csv'))]
#     print('Number of grids =',len(files))
    df_list = [pd.read_csv(join(folder, f)).drop(['system:index','.geo'],axis=1) for f in files]
    df = pd.concat(df_list, axis=0)
    
    return df

In [5]:
df_list = []

for year in years:
#     print(year)
    folder = os.path.join(repo,'data','raw','harmonics','GEE_usCrop_tableExports_gridHarmonics_{}'.format(year))
    df = init_df(folder)
    df_list.append(df)
    
df_harm = pd.concat(df_list, axis=0)
print(df_harm.shape)
df_harm.head()

(5000, 106)


/home/sditom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,BLUE_constant,BLUE_cos,BLUE_cos2,BLUE_sin,BLUE_sin2,CRC_constant,CRC_cos,CRC_cos2,CRC_sin,CRC_sin2,...,WDRVI_sin2,confidence,cropland,fips5,gridID,lat,lon,state,uniqueID,year
0,0.046021,-0.018157,0.006815,0.010658,-0.006999,0.411536,0.081220,-0.034986,-0.019757,-0.014701,...,0.103187,NaN,NaN,26001,195.0,44.831768,-83.572724,MI,0195_183,1999
1,0.042214,-0.011021,-0.000736,0.010584,-0.006501,0.478942,0.063167,0.023490,-0.079617,0.043724,...,0.099910,NaN,NaN,26007,195.0,44.867980,-83.572397,MI,0195_003,1999
2,0.035574,-0.008494,-0.000633,0.006224,-0.001837,0.374327,0.085324,-0.001563,-0.076721,0.023283,...,0.018507,NaN,NaN,26007,195.0,44.899742,-83.572895,MI,0195_235,1999
3,0.071882,-0.024716,0.015248,0.000128,0.005740,0.443246,0.044724,-0.003787,-0.015332,0.033716,...,-0.192097,NaN,NaN,26007,195.0,44.955885,-83.543265,MI,0195_043,1999
4,0.075048,-0.032080,-0.002132,0.021299,-0.003246,0.465383,0.090958,0.044347,-0.052491,0.011710,...,-0.138408,NaN,NaN,26007,195.0,44.960926,-83.548727,MI,0195_015,1999


#### Create a clean version: drop nan in Cropland and Harmonics columns

In [6]:
## NANs are due to:
# - cropland column ( for years/state where we don't have CDL)
# - confidence (for CDL years <2008 we don't have this information)
# - harmonics are null due to Clouds

In [7]:
df_harm_clean = df_harm.dropna(subset = ['cropland','BLUE_constant'])
print(df_harm_clean.shape)

(2907, 106)


### 2) Weather

In [8]:
def init_df2(folder, years):
    
    files = [f for f in listdir(folder) if (isfile(join(folder, f)) and f.endswith('.csv') and years in f)]
#     print('Number of grids =',len(files))
    df_list = [pd.read_csv(join(folder, f)).drop(['system:index','.geo'],axis=1) for f in files]
    df = pd.concat(df_list, axis=0)
    
    return df

In [9]:
folder = os.path.join(repo,'data','raw','weather')
df_1999_2009 = init_df2(folder,'1999-2009')
df_2010_2017 = init_df2(folder,'2010-2017')
df_2018_2018 = init_df2(folder,'2018-2018')
print(df_1999_2009.shape)
print(df_2010_2017.shape)
print(df_2018_2018.shape)

(250, 226)
(250, 166)
(250, 26)


In [10]:
admin =['uniqueID', 'lat', 'lon', 'gridID','state','fips5']

In [11]:
wvars = ['GDD_ss','aridity', 'ppt_aug', 'ppt_jul', 'ppt_jun', 'pr_early', 'pr_grow', 'tc_def_jul', 'tc_def_may', 
         'tc_soilm_aug', 'tmax_aug', 'tmax_jul', 'tmax_jun', 'tmax_may',
         'tmin_aug', 'tmin_jul', 'tmin_jun', 'tmin_may', 'vpd_july', 'vpd_jun']
len(wvars)# 20

20

In [12]:
df_1999_2009long = pd.wide_to_long(df_1999_2009, stubnames=wvars, i=admin, j="year", sep='_').reset_index()

In [13]:
df_2010_2017long = pd.wide_to_long(df_2010_2017, stubnames=wvars, i=admin, j="year", sep='_').reset_index()

In [14]:
df_2018_2018long = pd.wide_to_long(df_2018_2018, stubnames=wvars, i=admin, j="year", sep='_').reset_index()

In [15]:
df_weather = pd.concat([df_1999_2009long,df_2010_2017long,df_2018_2018long])
print(df_weather.shape)
df_weather.head()

(5000, 27)


,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,tmax_aug,tmax_jul,tmax_jun,tmax_may,tmin_aug,tmin_jul,tmin_jun,tmin_may,vpd_july,vpd_jun
0,0195_183,44.831768,-83.572724,195.0,MI,26001,1999,1066,0.585143,45.575976,...,24.644342,28.858575,25.827814,20.728662,11.494348,15.479761,11.933740,5.911493,1.067000,0.972536
1,0195_183,44.831768,-83.572724,195.0,MI,26001,2000,892,0.580832,87.352767,...,24.242761,25.242456,22.889764,19.708093,11.421716,11.936212,10.143335,6.120081,0.870705,0.760322
2,0195_183,44.831768,-83.572724,195.0,MI,26001,2001,1040,0.439536,83.356549,...,27.488000,26.529626,24.936731,19.997308,13.815698,12.633386,11.477350,7.350153,1.056728,0.907964
3,0195_183,44.831768,-83.572724,195.0,MI,26001,2002,1009,0.593639,85.388468,...,25.788660,28.905420,23.336420,15.644250,12.560236,15.284723,11.501123,3.155603,1.156021,0.768201
4,0195_183,44.831768,-83.572724,195.0,MI,26001,2003,908,0.524477,49.278941,...,26.430688,26.521783,23.739587,16.448175,13.374567,12.941187,9.005914,4.689508,0.922876,0.892054


#### Create a weather df clean version - with no missing data

In [16]:
df_weather_clean = df_weather.dropna()
df_weather_clean.shape

(5000, 27)

### 3) Merge Harmonics and Weather

In [32]:
cols = ['uniqueID', 'lat', 'lon', 'gridID','state','fips5','year']
df = df_weather_clean.merge(df_harm_clean, on=cols)
print(df.shape)
df.head()

(2907, 126)


,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,TVI_cos2,TVI_sin,TVI_sin2,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland
0,0195_183,44.831768,-83.572724,195.0,MI,26001,2007,1050,0.450314,63.393058,...,-0.415817,-3.413686,3.034775,0.002887,0.267021,-0.015529,-0.134148,0.207720,NaN,143.0
1,0195_183,44.831768,-83.572724,195.0,MI,26001,2008,944,0.719692,58.930219,...,-1.446181,-3.679035,0.385085,-0.061788,0.398413,-0.081385,-0.214360,0.150665,72.0,190.0
2,0195_183,44.831768,-83.572724,195.0,MI,26001,2009,764,0.820560,124.148264,...,5.408179,-15.419037,-9.652534,-0.034875,0.288924,0.043655,-0.328673,-0.025822,78.0,190.0
3,0195_003,44.867980,-83.572397,195.0,MI,26007,2007,1090,0.432146,62.766218,...,-0.839637,-8.215668,2.906648,0.067314,0.392029,-0.070124,-0.308558,0.188941,NaN,141.0
4,0195_003,44.867980,-83.572397,195.0,MI,26007,2008,989,0.681540,55.882887,...,-1.123797,-7.371482,2.499156,0.007580,0.416110,-0.013347,-0.377657,0.085755,68.0,141.0


### 4) Filter to crop points only

In [33]:
# subset df to only crop classes
df['cropbinary'] = (((df['cropland'] <= 60) | (df['cropland'] >= 196)) | ((df['cropland'] >= 66) & (df['cropland'] <= 77)))
df = df[df['cropbinary']==True]
df = df.drop('cropbinary',axis=1)
print(df.shape)
df.head()

(310, 126)


,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,TVI_cos2,TVI_sin,TVI_sin2,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland
9,0195_043,44.955885,-83.543265,195.0,MI,26007,2007,1035,0.457851,58.781680,...,0.475868,-0.646500,4.901464,-0.274471,-0.037866,0.024500,-0.030457,0.255380,NaN,24.0
10,0195_043,44.955885,-83.543265,195.0,MI,26007,2008,937,0.713359,50.110054,...,2.190074,-7.837471,-4.366203,-0.240506,0.339154,0.063453,-0.229946,-0.119806,42.0,1.0
11,0195_043,44.955885,-83.543265,195.0,MI,26007,2009,760,0.891211,141.505923,...,3.637766,-2.824670,-0.361237,-0.244305,0.448010,0.200878,-0.082083,-0.056604,100.0,1.0
13,0195_015,44.960926,-83.548727,195.0,MI,26007,2008,942,0.703785,50.019540,...,-1.697778,-6.221075,0.534639,-0.317568,0.136364,-0.112246,-0.254434,0.003037,37.0,28.0
14,0195_015,44.960926,-83.548727,195.0,MI,26007,2009,767,0.884511,143.313042,...,1.487675,-5.379326,2.194562,-0.235340,0.115122,0.042514,-0.196072,0.072877,37.0,24.0


## Prepare df 

### 5) Add CDL - target column (0=other crops, 1= corn , 5=soy)

In [34]:
df['CDL'] = 0
df.loc[df['cropland'] == 1, 'CDL'] = 1
df.loc[df['cropland'] == 5, 'CDL'] = 5
df.head()

,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,TVI_sin,TVI_sin2,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland,CDL
9,0195_043,44.955885,-83.543265,195.0,MI,26007,2007,1035,0.457851,58.781680,...,-0.646500,4.901464,-0.274471,-0.037866,0.024500,-0.030457,0.255380,NaN,24.0,0
10,0195_043,44.955885,-83.543265,195.0,MI,26007,2008,937,0.713359,50.110054,...,-7.837471,-4.366203,-0.240506,0.339154,0.063453,-0.229946,-0.119806,42.0,1.0,1
11,0195_043,44.955885,-83.543265,195.0,MI,26007,2009,760,0.891211,141.505923,...,-2.824670,-0.361237,-0.244305,0.448010,0.200878,-0.082083,-0.056604,100.0,1.0,1
13,0195_015,44.960926,-83.548727,195.0,MI,26007,2008,942,0.703785,50.019540,...,-6.221075,0.534639,-0.317568,0.136364,-0.112246,-0.254434,0.003037,37.0,28.0,0
14,0195_015,44.960926,-83.548727,195.0,MI,26007,2009,767,0.884511,143.313042,...,-5.379326,2.194562,-0.235340,0.115122,0.042514,-0.196072,0.072877,37.0,24.0,0


### 6) Drop points at border for state-year with no CDL

In [35]:
points_count = df.groupby(['state','year']).uniqueID.count().reset_index().rename({'uniqueID':'npoints'},axis=1)
df = df.merge(points_count, on=['state','year'], how = 'left')
df = df[df['npoints']>5]# will drop 10 rows: 1205148-1205138
df = df.drop('npoints',axis=1)
print(df.shape)
df.head()

(310, 127)


,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,TVI_sin,TVI_sin2,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland,CDL
0,0195_043,44.955885,-83.543265,195.0,MI,26007,2007,1035,0.457851,58.781680,...,-0.646500,4.901464,-0.274471,-0.037866,0.024500,-0.030457,0.255380,NaN,24.0,0
1,0195_043,44.955885,-83.543265,195.0,MI,26007,2008,937,0.713359,50.110054,...,-7.837471,-4.366203,-0.240506,0.339154,0.063453,-0.229946,-0.119806,42.0,1.0,1
2,0195_043,44.955885,-83.543265,195.0,MI,26007,2009,760,0.891211,141.505923,...,-2.824670,-0.361237,-0.244305,0.448010,0.200878,-0.082083,-0.056604,100.0,1.0,1
3,0195_015,44.960926,-83.548727,195.0,MI,26007,2008,942,0.703785,50.019540,...,-6.221075,0.534639,-0.317568,0.136364,-0.112246,-0.254434,0.003037,37.0,28.0,0
4,0195_015,44.960926,-83.548727,195.0,MI,26007,2009,767,0.884511,143.313042,...,-5.379326,2.194562,-0.235340,0.115122,0.042514,-0.196072,0.072877,37.0,24.0,0


#### Add state_name and state_abbr columns

In [36]:
abbr_to_state = {'IL':'Illinois', 'IA':'Iowa', 'IN':'Indiana', 'NE':'Nebraska', 'ND':'North Dakota',
                 'SD':'South Dakota', 'MN':'Minnesota', 'WI':'Wisconsin', 'MI':'Michigan',
                 'KS':'Kansas','KY':'Kentucky', 'OH':'Ohio', 'MO':'Missouri'}

df['state_abbrs'] = df['state']
df["state_name"] = df["state"].replace(abbr_to_state)
print(df.shape)
df.head()

(310, 129)


,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland,CDL,state_abbrs,state_name
0,0195_043,44.955885,-83.543265,195.0,MI,26007,2007,1035,0.457851,58.781680,...,-0.274471,-0.037866,0.024500,-0.030457,0.255380,NaN,24.0,0,MI,Michigan
1,0195_043,44.955885,-83.543265,195.0,MI,26007,2008,937,0.713359,50.110054,...,-0.240506,0.339154,0.063453,-0.229946,-0.119806,42.0,1.0,1,MI,Michigan
2,0195_043,44.955885,-83.543265,195.0,MI,26007,2009,760,0.891211,141.505923,...,-0.244305,0.448010,0.200878,-0.082083,-0.056604,100.0,1.0,1,MI,Michigan
3,0195_015,44.960926,-83.548727,195.0,MI,26007,2008,942,0.703785,50.019540,...,-0.317568,0.136364,-0.112246,-0.254434,0.003037,37.0,28.0,0,MI,Michigan
4,0195_015,44.960926,-83.548727,195.0,MI,26007,2009,767,0.884511,143.313042,...,-0.235340,0.115122,0.042514,-0.196072,0.072877,37.0,24.0,0,MI,Michigan


### 7) Save final df

In [ ]:
# df.to_csv(os.path.join(repo,'data','raw','alldata_1999_2018.csv'),index=False)